In [2]:
import utility.stock

data = utility.stock.get_stock_history('AAPL','1y')

[*********************100%***********************]  1 of 1 completed


In [5]:
import pandas as pd

period = 14

rsi = pd.DataFrame()
rsi['close'] = data['Close']
rsi = rsi.reset_index()
rsi = rsi.rename(columns={"Date": "date"})

if len(rsi.index) <= period:
    print("Error: Data size too small")

rsi['close_shift'] = rsi['close'].shift(1)
rsi['gain'] = rsi['close'] - rsi['close_shift']
rsi['loss'] = rsi['close_shift'] - rsi['close']
rsi['gain'] = rsi['gain'].clip(lower=0)
rsi['loss'] = rsi['loss'].clip(lower=0)

rsi.loc[period,'avg_gain'] = rsi['gain'].iloc[1:period+1].mean()
rsi.loc[period,'avg_loss'] = rsi['loss'].iloc[1:period+1].mean()

for i in range(period + 1, len(rsi)):
    rsi.loc[i,'avg_gain'] = (rsi.loc[i-1,'avg_gain'] * (period -1) + rsi.loc[i,'gain'])/ period
    rsi.loc[i,'avg_loss'] = (rsi.loc[i-1,'avg_loss'] * (period -1) + rsi.loc[i,'loss'])/ period

rsi['rs'] = rsi['avg_gain']/rsi['avg_loss']
rsi['rsi'] = 100-(100/(1 + rsi['rs']))

rsi = rsi.set_index('date')
#rsi = rsi.dropna()
#rsi = rsi.drop(columns=['close', 'close_shift', 'gain','loss','avg_gain','avg_loss','rs'])

print(rsi)
rsi.to_csv('test2.csv')

                 close  close_shift      gain      loss  avg_gain  avg_loss  \
date                                                                          
2022-09-09  157.369995          NaN       NaN       NaN       NaN       NaN   
2022-09-12  163.429993   157.369995  6.059998  0.000000       NaN       NaN   
2022-09-13  153.839996   163.429993  0.000000  9.589996       NaN       NaN   
2022-09-14  155.309998   153.839996  1.470001  0.000000       NaN       NaN   
2022-09-15  152.369995   155.309998  0.000000  2.940002       NaN       NaN   
...                ...          ...       ...       ...       ...       ...   
2023-09-01  189.460007   187.869995  1.590012  0.000000  1.258792  0.773167   
2023-09-05  189.699997   189.460007  0.239990  0.000000  1.186021  0.717941   
2023-09-06  182.910004   189.699997  0.000000  6.789993  1.101305  1.151659   
2023-09-07  177.559998   182.910004  0.000000  5.350006  1.022640  1.451541   
2023-09-08  178.179993   177.559998  0.619995  0.000